# Upload Edge Impulse model zip → Vela conversion → Labels → Manifest download

This Colab notebook lets you upload a zipped Edge Impulse model named `<modelname>-custom-<version>.zip`, converts the embedded `trained.tflite` with Arm Vela, extracts labels from `model-parameters/model_variables.h`, and packages both into a `Manifest` folder ready to download.

What you’ll get:
- `<modelname>-custom-<version>_vela.tflite` (Vela-optimized model)
- `labels.txt` (classes extracted from the model)
- A downloadable `Manifest.zip` containing both files

Naming convention example:
- `wildlife-watcher-rat-classifcation-model-custom-v10.zip`

Run the cells in order. If a step fails, read the note above that step for quick fixes.

In [ ]:
# Install Arm Vela CLI in Colab
!pip install ethos-u-vela

# Step 1: Setup (imports, helpers)
# - Installs/validates Arm Vela
# - Imports core libraries
# - Defines small helpers for name parsing and safe file ops

import os
import re
import zipfile
import shutil
from pathlib import Path

# Ensure paths use /content in Colab
BASE = Path('/content')
BASE.mkdir(parents=True, exist_ok=True)

# Try to ensure Vela CLI is available
try:
    import subprocess
    _ = subprocess.run(['vela', '--version'], capture_output=True, text=True)
    if _.returncode != 0:
        raise RuntimeError('Vela not found')
except Exception:
    # Vela is preinstalled on many Colab images; if not, guide the user.
    print('NOTE: If vela is missing, run: !pip install ethos-u-vela')


def parse_model_zip_name(zip_path: str):
    """Parse '<modelname>-custom-<version>.zip' -> (modelname, version)
    Raises ValueError if the pattern does not match.
    """
    name = os.path.basename(zip_path)
    if not name.endswith('.zip'):
        raise ValueError('Zip file must end with .zip')
    base = name[:-4]
    if '-custom-' not in base:
        raise ValueError("Filename must contain '-custom-' (e.g. mymodel-custom-v10.zip)")
    modelname, version = base.split('-custom-', 1)
    if not modelname or not version:
        raise ValueError('Invalid filename segments before/after -custom-')
    return modelname, version


def safe_move(src: Path, dst: Path):
    dst.parent.mkdir(parents=True, exist_ok=True)
    if dst.exists():
        dst.unlink()
    shutil.move(str(src), str(dst))

print('Setup complete. Proceed to Step 2 to upload your zip.')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.3 MB/s eta 0:00:00
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 25.9.23
    Uninstalling flatbuffers-25.9.23:
      Successfully uninstalled flatbuffers-25.9.23
Setup complete. Proceed to Step 2 to upload your zip.


## Step 2: Upload your Edge Impulse model zip

- Upload a file named `<modelname>-custom-<version>.zip` (e.g., `wildlife-watcher-rat-classifcation-model-custom-v10.zip`).
- The zip should contain `trained.tflite` and a `model-parameters/model_variables.h` file.
- After upload, the next cells will unzip, convert with Vela, extract labels, and package outputs.

If you see a filename error, rename the file locally first and re-upload.

In [ ]:
# Step 2 (run): Upload your zip
# - Click the file picker and choose a file named <modelname>-custom-<version>.zip
# - We'll save it under /content/uploads

from pathlib import Path

try:
    from google.colab import files as colab_files  # type: ignore
    IS_COLAB = True
except Exception:
    IS_COLAB = False

UPLOADS = BASE / 'uploads'
UPLOADS.mkdir(parents=True, exist_ok=True)

if IS_COLAB:
    print('Pick your <modelname>-custom-<version>.zip...')
    uploaded = colab_files.upload()  # returns dict: name -> bytes
    if not uploaded:
        raise RuntimeError('No file uploaded')
    up_name = list(uploaded.keys())[0]
    uploaded_zip_path = UPLOADS / up_name
    with open(uploaded_zip_path, 'wb') as f:
        f.write(uploaded[up_name])
else:
    # Not running in Colab: set uploaded_zip_path manually to an existing zip on disk
    # Example: uploaded_zip_path = UPLOADS / 'wildlife-watcher-rat-classifcation-model-custom-v10.zip'
    raise EnvironmentError('Not running in Colab. Set uploaded_zip_path to a local path before proceeding.')

print('Uploaded to:', uploaded_zip_path)

Pick your <modelname>-custom-<version>.zip...


Saving wildlife-watcher-rat-classifcation-model-custom-v10.zip to wildlife-watcher-rat-classifcation-model-custom-v10.zip
Uploaded to: /content/uploads/wildlife-watcher-rat-classifcation-model-custom-v10.zip


## Step 3: Unzip model and validate contents

This will:
- Parse the filename to extract `<modelname>` and `<version>`
- Unzip the archive to `/content/work/<modelname>-custom-<version>`
- Verify that `trained.tflite` exists
- Verify that `model-parameters/model_variables.h` exists

If validation fails, fix the zip structure in Edge Impulse and re-export.

In [ ]:
# Step 3 (run): Unzip and validate

import zipfile

# Parse the filename
model_name, model_version = parse_model_zip_name(str(uploaded_zip_path))
container_name = f"{model_name}-custom-{model_version}"

WORK = BASE / 'work' / container_name
if WORK.exists():
    shutil.rmtree(WORK)
WORK.mkdir(parents=True, exist_ok=True)

# Unzip under work container
with zipfile.ZipFile(uploaded_zip_path, 'r') as z:
    z.extractall(WORK)

# Locate files
TFLITE = WORK / 'trained.tflite'
VARS_H = WORK / 'model-parameters' / 'model_variables.h'

if not TFLITE.exists():
    raise FileNotFoundError(f"trained.tflite not found at {TFLITE}")
if not VARS_H.exists():
    raise FileNotFoundError(f"model_variables.h not found at {VARS_H}")

print('Container:', container_name)
print('trained.tflite:', TFLITE)
print('model_variables.h:', VARS_H)

Container: wildlife-watcher-rat-classifcation-model-custom-v10
trained.tflite: /content/work/wildlife-watcher-rat-classifcation-model-custom-v10/trained.tflite
model_variables.h: /content/work/wildlife-watcher-rat-classifcation-model-custom-v10/model-parameters/model_variables.h


## Step 4: Convert with Arm Vela (Ethos-U)

This will run `vela` on `trained.tflite` and produce a Vela-optimized model named `<modelname>-custom-<version>_vela.tflite`.

Notes:
- If `vela` is missing, run the install hint from Step 1.

In [ ]:
# Step 4 (run): Vela conversion

import subprocess

VELA_OUT = WORK / f"{container_name}_vela.tflite"

# HX6538 WiseEye 2 uses ARM Cortex-M55 + Ethos-U55-64 NPU
cmd = [
    'vela',
    '--accelerator-config', 'ethos-u55-64',  # HX6538 has Ethos-U55 with 64 MACs/cycle
    '--memory-mode', 'Shared_Sram',         # typical mode for WiseEye 2
    '--output-dir', str(WORK),
    str(TFLITE),
]

print('Running:', ' '.join(cmd))
res = subprocess.run(cmd, capture_output=True, text=True)
print(res.stdout)
if res.returncode != 0:
    print(res.stderr)
    raise RuntimeError('Vela conversion failed')

# Vela may write to same filename or suffix _vela.tflite; ensure final path
# If WORK/trained_vela.tflite exists, move/rename to expected name
possible = [
    WORK / 'trained_vela.tflite',
    WORK / 'trained.tflite',
    WORK / f'{TFLITE.stem}_vela.tflite',
]
produced = None
for p in possible:
    if p.exists() and p != TFLITE:
        produced = p
        break

if produced is None:
    # Some Vela versions overwrite in-place; copy with new name
    produced = TFLITE

# safe_move(produced, VELA_OUT) # Original line
safe_move(produced, WORK / 'MOD00001.tfl') # Modified line
VELA_OUT = WORK / 'MOD00001.tfl' # Update the VELA_OUT variable with the new filename

print('Vela model:', VELA_OUT)

Running: vela --accelerator-config ethos-u55-64 --memory-mode Shared_Sram --output-dir /content/work/wildlife-watcher-rat-classifcation-model-custom-v10 /content/work/wildlife-watcher-rat-classifcation-model-custom-v10/trained.tflite

Network summary for trained
Accelerator configuration                Ethos_U55_64
System configuration             Ethos_U55_High_End_Embedded
Memory mode                               Shared_Sram
Accelerator clock                                 500 MHz
Design peak SRAM bandwidth                       3.73 GB/s
Design peak Off-chip Flash bandwidth             0.47 GB/s

Total SRAM used                                 18.27 KiB
Total Off-chip Flash used                       69.91 KiB

CPU operators = 0 (0.0%)
NPU operators = 60 (100.0%)

Average SRAM bandwidth                           0.95 GB/s
Input   SRAM bandwidth                           0.06 MB/batch
Weight  SRAM bandwidth                           0.06 MB/batch
Output  SRAM bandwidth             

## Step 5: Extract labels from model_variables.h

This will read `model-parameters/model_variables.h` and extract the classifier categories using a regex. The labels will be saved to `labels.txt` (one per line).

In [ ]:
# Step 5 (run): Extract labels and write labels.txt

# Read model_variables.h and parse category array
with open(VARS_H, 'r') as f:
    content = f.read()

match = re.search(r'const char\* ei_classifier_inferencing_categories.*?=\s*\{(.*?)\};', content, re.DOTALL)
if match:
    labels = re.findall(r'"([^"]+)"', match.group(1))
else:
    labels = []

if not labels:
    raise RuntimeError('No labels found in model_variables.h. Check the file format or regex.')

LABELS_TXT = WORK / 'labels.txt'
with open(LABELS_TXT, 'w') as f:
    f.write('\n'.join(labels))

print('Labels:', labels)
print('labels.txt:', LABELS_TXT)

Labels: ['not rat', 'rat']
labels.txt: /content/work/wildlife-watcher-rat-classifcation-model-custom-v10/labels.txt


## Step 6: Package Manifest and download

This will:
- Create a `Manifest` folder under the work container.
- Copy the Vela-converted model and `labels.txt` into it.
- Create `Manifest.zip` and provide a download link/button in Colab.

In [ ]:
# Step 6 (run): Create Manifest and provide download

from shutil import copy2, make_archive

MANIFEST_DIR = WORK / 'Manifest'
MANIFEST_DIR.mkdir(parents=True, exist_ok=True)

# Copy outputs
# Ensure VELA_OUT is a Path object before accessing .name
if not isinstance(VELA_OUT, Path):
    VELA_OUT = Path(VELA_OUT)

# Copy outputs
copy2(VELA_OUT, MANIFEST_DIR / VELA_OUT.name)
copy2(LABELS_TXT, MANIFEST_DIR / 'labels.txt')

# Zip Manifest
manifest_zip_base = str(WORK / 'Manifest')
manifest_zip_path = f"{manifest_zip_base}.zip"
# Remove existing zip if present
if os.path.exists(manifest_zip_path):
    os.remove(manifest_zip_path)
make_archive(manifest_zip_base, 'zip', root_dir=WORK, base_dir='Manifest')

print('Manifest folder:', MANIFEST_DIR)
print('Manifest.zip:', manifest_zip_path)

# Provide download (Colab)
if IS_COLAB:
    from google.colab import files as colab_files  # type: ignore
    colab_files.download(manifest_zip_path)

Manifest folder: /content/work/wildlife-watcher-rat-classifcation-model-custom-v10/Manifest
Manifest.zip: /content/work/wildlife-watcher-rat-classifcation-model-custom-v10/Manifest.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>